In [ ]:
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import register_all_modules
from mmdet.registry import VISUALIZERS
import mmcv

In [ ]:
# download the checkpoint demo
work_dir = "../work_dirs/convnext/"
config_file = work_dir + 'convnext.py'
checkpoint_file = work_dir + 'epoch_2.pth'


In [ ]:
#Register all modules in mmdet into the registries
register_all_modules()
# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')  # or device='cpu'

In [ ]:
import os
from typing import List

def get_image_files(directory: str) -> List[str]:
    image_files = []
    if not os.path.isdir(directory):
        return image_files

    for filename in os.listdir(directory):
        if filename.lower().endswith((".png", ".jpg")):
            image_files.append(os.path.join(directory, filename))

    return image_files

def get_png_files_from_directory(base_directory: str, directory_numbers: int) -> List[str]:

    directory_path = os.path.join(base_directory, f"00000{directory_numbers}/rgb")
    png_files = get_image_files(directory_path)
    png_files.sort()

    return png_files

In [ ]:
import numpy as np
import pycocotools.mask as mask_util
def results_to_coco_segm(results, scene_id):
    coco_segm = []
    for img_id, result in enumerate(results):
        instances = result.pred_instances
        
        for bbox, label, score, mask in zip(instances.bboxes, instances.labels, instances.scores, instances.masks):     
                
            bbox = bbox.tolist()
            label = label.item() + 1 #
            score = score.item()
            mask = mask.cpu().numpy()
                
            segmentation = mask_util.encode(np.array(mask[:, :, np.newaxis], order='F', dtype='uint8'))[0]
            
            bbox = [
                bbox[0],
                bbox[1],
                bbox[2] - bbox[0],
                bbox[3] - bbox[1],
            ]
            annotation = {
                "scene_id": scene_id,
                "image_id": img_id,
                "category_id": label,
                "score": score,
                "bbox": bbox,
                "time": -1,
                "segmentation": segmentation
            }       
            coco_segm.append(annotation) 
            
    return coco_segm

In [ ]:
base_directory = "../data/ic_bin_test/"

final_segm = [] 
directory_numbers = [1, 2, 3]
for dir_number in directory_numbers:
    image_files = get_png_files_from_directory(base_directory, dir_number)
    inference_results = inference_detector(model, image_files)
    coco_segm = results_to_coco_segm(inference_results, dir_number )
    final_segm.extend(coco_segm)

In [ ]:
import json

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            return obj.decode('utf-8')  # Decode byte string to UTF-8
        return super().default(obj)

output_file = "/home/hans/Desktop/BOP/results/segm_icbin-test.json"

with open(output_file, "w") as json_file:
    json.dump(final_segm, json_file, cls=CustomJSONEncoder)

print("Data saved to", output_file)